In [ ]:
from google.colab import drive
#cargando libreria especializada de google
drive.mount('/content/drive')
# montar:

Mounted at /content/drive


In [ ]:
# TODO: Establece lenguas de origen y destino. Estos utilizan tradicionalmente los códigos de una lengua
#que se encuentran en JW300
# Los códigos serán los sufijos que llevaran los nombres los archivos que se generen

import os
#cargando os, acceso a comandis especializados del SO
source_language = "es"
target_language = "mco" 
lc = False  # If True, datos en minuscula
seed = 42  # Siembra semilla 42
tag = "baseline" # nombre de la carpeta donde se almacenarán los datos
os.environ["src"] = source_language # Para usar los nombre en bash
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# crea una carpeta en drive
!mkdir -p "/content/drive/My Drive/mixe-coatlan-es-mco/$src-$tgt-$tag"
os.environ["gdrive_path"] = "/content/drive/My Drive/mixe-coatlan-es-mco/%s-%s-%s" % (source_language, target_language, tag)

In [ ]:
!echo $gdrive_path
#IMPRIME EL PATH ANTES CREADO

/content/drive/My Drive/mixe-coatlan-es-mco/es-mco-baseline


In [ ]:
# Install opus-tools
! pip install opustools-pkg

     |████████████████████████████████| 81kB 4.0MB/s 


In [ ]:
# bajamos nuestro corpus disponible en jw300 con opus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# Descomprime los datos
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/es-mco.xml.gz not found. The following files are available for downloading:

 860 KB https://object.pouta.csc.fi/OPUS-JW300/v1b/xml/es-mco.xml.gz
 217 MB https://object.pouta.csc.fi/OPUS-JW300/v1b/xml/es.zip
   7 MB https://object.pouta.csc.fi/OPUS-JW300/v1b/xml/mco.zip

 225 MB Total size
./JW300_latest_xml_es-mco.xml.gz ... 100% of 860 KB
./JW300_latest_xml_es.zip ... 100% of 217 MB
./JW300_latest_xml_mco.zip ... 100% of 7 MB


In [ ]:
# Descarga un test global para inglés
#! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# Y un conjunto de pruebas para estas lenguas, pero no será utilizado ya que no existe test para ayuuk de coatlán ni para español
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2021-03-22 19:33:20--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-mco.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-03-22 19:33:20 ERROR 404: Not Found.

mv: cannot stat 'test.en-mco.en': No such file or directory
--2021-03-22 19:33:20--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-mco.mco
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-03-22 19:33:20 ERROR 404: Not Found.

mv: cannot stat 'test.en-mco.mco': No such file or 

In [ ]:
ls

drive/    JW300_latest_xml_es-mco.xml  JW300_latest_xml_mco.zip  sample_data/
jw300.es  JW300_latest_xml_es.zip      jw300.mco


In [ ]:
! cat jw300.es > drive/MyDrive/mixe-coatlan-es-mco/es-mco-baseline/es.txt
! cat jw300.mco > drive/MyDrive/mixe-coatlan-es-mco/es-mco-baseline/mco.txt

In [ ]:
import os
import pandas as pd
import numpy as np
import re

In [ ]:
#Los textos ya vienen alineados, solo es cuestion de unirlos y eliminar 
#las que no tuvieron correspondecia así como los duplicados
source = []
target = []

with open("drive/My Drive/mixe-coatlan-es-mco/es-mco-baseline/es.txt") as f:
    for i, line in enumerate(f):
        source.append(line.strip().lower())
        

with open("drive/My Drive/mixe-coatlan-es-mco/es-mco-baseline/mco.txt") as f:
    for i, line in enumerate(f):
        target.append(line.strip().lower())

print(len(source))
print(len(target))

79132
79132


In [ ]:
# junta source y target
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])

print(len(df))
print(" ")
df.head(25)

79132
 


,source_sentence,target_sentence
0,índice,tijaty myëmiimpy
1,1 de marzo de 2011,1 äämbë abril 2011
2,las “ buenas nuevas del reino ”,"“ oybyë ayuk ” mä ja anaˈamën , ¿ ti yˈandijpy ?"
3,un mensaje de esperanza serie de portada,yajnimaytyaˈaky diˈib miimp mä nyiˈak
4,3 una profecía de gran importancia,3 diˈib ojts yajnaskäjpxë mëjwiin kajaa nyikejy
5,4 ¿ qué es el reino de dios ?,4 ¿ ti yˈandijpy ja diosë yˈanaˈamën ?
6,6 ¿ qué son las buenas nuevas ?,6 ¿ ti yˈandijpy ja oybyë ayuk ?
7,7 ¿ quiénes están predicando las buenas nuevas ?,7 ¿ pënaty kyäjpxwäˈkxtëp ja oybyë ayuk ?
8,8 ¿ qué es “ el fin ” ?,8 ¿ ti duˈun yˈandijpy “ ja jëjptëgoˈoyën ” ?
9,secciones fijas,nanduˈun


In [ ]:
# elimina duplicados 
df_pp = df.drop_duplicates()

df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)
df_pp

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,source_sentence,target_sentence
0,índice,tijaty myëmiimpy
1,1 de marzo de 2011,1 äämbë abril 2011
2,las “ buenas nuevas del reino ”,"“ oybyë ayuk ” mä ja anaˈamën , ¿ ti yˈandijpy ?"
3,un mensaje de esperanza serie de portada,yajnimaytyaˈaky diˈib miimp mä nyiˈak
4,3 una profecía de gran importancia,3 diˈib ojts yajnaskäjpxë mëjwiin kajaa nyikejy
...,...,...
79127,encontraron un trabajo que les permite ser pre...,ojts tpäättë tyuunk diˈib mooyëdëbë tiempë par...
79128,"daniel dice : “ antes de irnos de españa , no ...",daniel jyënaˈany : “ mätsë nety ngatsoondënëm ...
79129,"hoy lo hacemos , y no nos falta nada verdadera...","tyam , adëyëts njukyˈattë ets nitiijëts ngayaj..."
79130,¿ cuál es su mayor alegría ?,¿ ti diˈib niˈigyë yajnayjäˈäwëdëp agujk jotku...


In [ ]:
es = df_pp[["source_sentence"]]
mco = df_pp[["target_sentence"]]

In [ ]:
es.to_csv("/content/drive/My Drive/mixe-coatlan-es-mco/es-mco-baseline/es-sin-dup.txt", header=False, index=False, sep = "\t")
mco.to_csv("/content/drive/My Drive/mixe-coatlan-es-mco/es-mco-baseline/mco-sin-dup.txt", header=False, index=False, sep = "\t")


# para el español

In [ ]:

def abre_archivo(nombre):
    with open(nombre) as archivo:
        return archivo.read()
    
#Elimina caracteres especiales del corpus
def limpia(a):
    entrada = ':"«»()[]—“”-;.|'
    salida  = '               '
    reemplaza = str.maketrans(entrada, salida)
    cadena_limpia = a.translate(reemplaza)
    return cadena_limpia

clean = abre_archivo("/content/drive/My Drive/mixe-coatlan-es-mco/es-mco-baseline/es-sin-dup.txt")
limpio = limpia(clean)


#Expresiones regulares utilizados para quitar abreviaciones de juan, lucas, mateo y marcos.
#Así como eliminar numeros en el corpus

eliminar = r'\b[0-9]{1,8}\b'
limpio = re.sub(eliminar, ' ', limpio)

# Para quitar espacios grandes entre palabras
limpio = limpio.replace("\n","|")
limpio = limpio.split()

def quita_espacio(lista):
    for i in range(len(lista)):
        lista[i]=lista[i].strip()
    return lista

limpio = quita_espacio(limpio)

def lista_a_cadena(lista):
    cadena = ' '.join(lista)
    return cadena

limpio = lista_a_cadena(limpio)

separaxpipe = limpio.split("|")
separaxpipe = [x.strip() for x in separaxpipe]

print(len(separaxpipe))

df_es = pd.DataFrame(separaxpipe, columns=["es"])
df_es

68837


,es
0,índice
1,de marzo de
2,las buenas nuevas del reino
3,un mensaje de esperanza serie de portada
4,una profecía de gran importancia
...,...
68832,"daniel dice antes de irnos de españa , no está..."
68833,"hoy lo hacemos , y no nos falta nada verdadera..."
68834,¿ cuál es su mayor alegría ?
68835,ellos comentan ayudar a las personas humildes ...


# Para mixe coatlán

In [ ]:
def abre_archivo(nombre):
    with open(nombre) as archivo:
        return archivo.read()
    
#Elimina caracteres especiales del corpus
def limpia(a):
    entrada = ':"«»()[]—“”-;.|áéíóú'
    salida  = '               aeiou'
    reemplaza = str.maketrans(entrada, salida)
    cadena_limpia = a.translate(reemplaza)
    return cadena_limpia

clean = abre_archivo("/content/drive/My Drive/mixe-coatlan-es-mco/es-mco-baseline/mco-sin-dup.txt")
limpio = limpia(clean)

#Eliminar numeros en el corpus
eliminar = r'\b[0-9]{1,8}\b'
limpio = re.sub(eliminar, ' ', limpio)

# Para quitar espacios grandes entre palabras
limpio = limpio.replace("\n","|")
limpio = limpio.split()

def quita_espacio(lista):
    for i in range(len(lista)):
        lista[i]=lista[i].strip()
    return lista

limpio = quita_espacio(limpio)

def lista_a_cadena(lista):
    cadena = ' '.join(lista)
    return cadena

limpio = lista_a_cadena(limpio)

separaxpipe = limpio.split("|")
separaxpipe = [x.strip() for x in separaxpipe]

print(len(separaxpipe))

df_mco = pd.DataFrame(separaxpipe, columns=["mco"])
df_mco

68837


,mco
0,tijaty myëmiimpy
1,äämbë abril
2,"oybyë ayuk mä ja anaˈamën , ¿ ti yˈandijpy ?"
3,yajnimaytyaˈaky diˈib miimp mä nyiˈak
4,diˈib ojts yajnaskäjpxë mëjwiin kajaa nyikejy
...,...
68832,daniel jyënaˈany mätsë nety ngatsoondënëm espa...
68833,"tyam , adëyëts njukyˈattë ets nitiijëts ngayaj..."
68834,¿ ti diˈib niˈigyë yajnayjäˈäwëdëp agujk jotku...
68835,yëˈëjëty jyënäˈändë nitiijëts duˈun xykyayajna...


In [ ]:
#Concatena en  mco
es_mco = pd.concat([df_es,df_mco], axis = 1)
es_mco = es_mco.dropna()
es_mco

,es,mco
0,índice,tijaty myëmiimpy
1,de marzo de,äämbë abril
2,las buenas nuevas del reino,"oybyë ayuk mä ja anaˈamën , ¿ ti yˈandijpy ?"
3,un mensaje de esperanza serie de portada,yajnimaytyaˈaky diˈib miimp mä nyiˈak
4,una profecía de gran importancia,diˈib ojts yajnaskäjpxë mëjwiin kajaa nyikejy
...,...,...
68832,"daniel dice antes de irnos de españa , no está...",daniel jyënaˈany mätsë nety ngatsoondënëm espa...
68833,"hoy lo hacemos , y no nos falta nada verdadera...","tyam , adëyëts njukyˈattë ets nitiijëts ngayaj..."
68834,¿ cuál es su mayor alegría ?,¿ ti diˈib niˈigyë yajnayjäˈäwëdëp agujk jotku...
68835,ellos comentan ayudar a las personas humildes ...,yëˈëjëty jyënäˈändë nitiijëts duˈun xykyayajna...


In [ ]:
df_pp = es_mco.drop_duplicates()

df_pp.drop_duplicates(subset='es', inplace=True)
df_pp.drop_duplicates(subset='mco', inplace=True)
df_pp

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,es,mco
0,índice,tijaty myëmiimpy
1,de marzo de,äämbë abril
2,las buenas nuevas del reino,"oybyë ayuk mä ja anaˈamën , ¿ ti yˈandijpy ?"
3,un mensaje de esperanza serie de portada,yajnimaytyaˈaky diˈib miimp mä nyiˈak
4,una profecía de gran importancia,diˈib ojts yajnaskäjpxë mëjwiin kajaa nyikejy
...,...,...
68831,encontraron un trabajo que les permite ser pre...,ojts tpäättë tyuunk diˈib mooyëdëbë tiempë par...
68832,"daniel dice antes de irnos de españa , no está...",daniel jyënaˈany mätsë nety ngatsoondënëm espa...
68833,"hoy lo hacemos , y no nos falta nada verdadera...","tyam , adëyëts njukyˈattë ets nitiijëts ngayaj..."
68834,¿ cuál es su mayor alegría ?,¿ ti diˈib niˈigyë yajnayjäˈäwëdëp agujk jotku...


In [ ]:
df_pp.to_csv("/content/drive/My Drive/mixe-coatlan-es-mco/es-mco-baseline/bitext-es-mco-todavia-sin-random.txt", header=False, index=False, sep = ".")

In [ ]:
# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

In [ ]:
df_pp.to_csv("/content/drive/My Drive/mixe-coatlan-es-mco/es-mco-baseline/corpus-train-dev-test-bitext-es-mco.txt", header=False, index=False, sep = ".")

In [ ]:
len(df_pp)

67197

In [ ]:
df_pp.head(10)

,es,mco
0,"si su esposo es humilde y modesto , no se sent...",duˈuntsoo dyaˈixëdët ko tsyejpyë nyëdoˈoxy ets...
1,siempre nos da justo lo que necesitamos para s...,pes xëmë xymyoˈoyëm diˈib meerë nyajtëgoyˈäjtë...
2,"si es así , ¡ bien hecho !",¡ jantsy oy pën ojts duˈun mduny !
3,la expresión un solo jehová significa varias c...,"ko jyënaˈany yëˈëyë tuˈugë jyobaa , kanäk pëky..."
4,pero cuando luego recibió información más exac...,"per ko tnijäˈäwë extëm tyëyˈäjtënëty , ta wyin..."
5,sabe qué ropa y qué comida necesitaremos,nyijäˈäwëp ti wit xox ets ti käˈäy ukën nyajtë...
6,no llegará tarde,tim tsojk jyäˈtäˈäny
7,nuestras publicaciones la usan para enseñarnos...,mä ëxpëjkpajn dyajtunyë tyäˈädë ijxpajtën parë...
8,"por otro lado , los esposos las dominarían y e...","ets ko yëˈë anaˈamanëdëp , axtë nääk tsiigan k..."
9,"además , puede que empecemos a ver las buenas ...",nan mbäädë net nˈijxëm ja oybyë jyaˈayˈäjtën d...


# Extraer 1000 pares para el test

In [ ]:
#Se escoge las primeras 912 frases para el corpus de test
test = df_pp.iloc[0:1000]
test = test.reset_index(drop=True)
test

,es,mco
0,"si su esposo es humilde y modesto , no se sent...",duˈuntsoo dyaˈixëdët ko tsyejpyë nyëdoˈoxy ets...
1,siempre nos da justo lo que necesitamos para s...,pes xëmë xymyoˈoyëm diˈib meerë nyajtëgoyˈäjtë...
2,"si es así , ¡ bien hecho !",¡ jantsy oy pën ojts duˈun mduny !
3,la expresión un solo jehová significa varias c...,"ko jyënaˈany yëˈëyë tuˈugë jyobaa , kanäk pëky..."
4,pero cuando luego recibió información más exac...,"per ko tnijäˈäwë extëm tyëyˈäjtënëty , ta wyin..."
...,...,...
995,"todos los hermanos , hasta los que tienen muy ...","niˈamukë nmëguˈukˈäjtëm , axtë diˈib kyaj nëgo..."
996,pongamos dos ejemplos supongamos que estamos l...,"extëm nˈokpëjtakëm , ¿ ti mbäät nduˈunëm ko my..."
997,¿ conoce usted a otros hermanos mayores que ha...,¿ miimp mä mwinmäˈäny tuˈugë nmëguˈukˈäjtëm di...
998,al ver que por todas partes había violencia y ...,noe mbäätxyëp tyuktëjkë moˈon tujkën ko tˈijxy...


In [ ]:
#Se extrae test por columna
es = test["es"]
mco = test["mco"]

es.to_csv('drive/My Drive/mixe-coatlan-es-mco/es-mco-baseline/test/test-random_es.txt', header=False, index=False, sep = "\t")
mco.to_csv('drive/My Drive/mixe-coatlan-es-mco/es-mco-baseline/test/test-random_mco.txt', header=False, index=False, sep = "\t")


In [ ]:
#Se eliminan las 1000 filas bitext de donde fueron extraídas las frases para test
df_pp = df_pp.drop(range(0,1000),axis = 0)
df_pp = df_pp.reset_index(drop=True)

df_pp

,es,mco
0,"sin duda , estos hermanos necesitan que siempr...",seguurë ko tyäˈädë nmëguˈukˈäjtëm yajtëgoyˈäjt...
1,toda escritura es inspirada de dios y provecho...,dios wyinmäˈänymyooy ja jyaˈayëty es tkëxjäˈäy...
2,el amor y unidad que se ve entre los testigos ...,pes diˈibë jesukristë xytyukˈanaˈamëm axtë jaw...
3,examinarlos con detalle nos permitirá conocer ...,"ko nˈëxpëjkëm yajxon , ta niˈigyë nnijawëyäˈän..."
4,"o , lo que es lo mismo , impedirá que puedan h...",o diˈib yˈandijpy ko mbäät tpëjkë myëjää parë ...
...,...,...
66192,"en el parís de la época de langton , vivían es...","ja tiempë mä jyukyˈajtyë langton jap paris , t..."
66193,¿ qué hará si pierde el trabajo ?,¿ ti tyunëp ko ja tyuunk yajpëjkët ?
66194,jehová les dio su espíritu santo a los primero...,jyobaa myëjämooy pënaty pyanëjkxtë kristë mä p...
66195,algo que nos ayudará a comprender los retos im...,tsiptakp nyajtuˈukmujkëmë jäˈäy diˈib wiˈixëmë...


In [ ]:
#Se extrae columnas para el corpus dev-train
train_dev_es = df_pp["es"]
train_dev_mco = df_pp["mco"]

train_dev_es.to_csv('drive/My Drive/mixe-coatlan-es-mco/es-mco-baseline/train-dev-random_es.txt', header=False, index=False, sep = "\t")
train_dev_mco.to_csv('drive/My Drive/mixe-coatlan-es-mco/es-mco-baseline/train-dev-random_mco.txt', header=False, index=False, sep = "\t")

In [ ]:
### Filtra para obtener train y dev

In [ ]:
# lee los datos de train para hacer un filtro de posibles repeticiones dentro del corpus de train y dev
en_test_sents = set()
filter_test_sents = "drive/My Drive/mixe-coatlan-es-mco/es-mco-baseline/test/test-random_es.txt"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))
#en_test_sents

Loaded 1000 global test sentences to filter from the training/dev data.


In [ ]:
print(en_test_sents)

{'pablo dijo que pensáramos los unos en los otros para incitarnos al amor y a las obras excelentes', 'en abril de , las autoridades empezaron a deportar a siberia a los testigos que vivían en el oeste de la unión soviética', 'quizás al principio debamos dar a los hermanos comida , ropa y otras cosas básicas * vea la nota', 'a pesar de todas estas situaciones difíciles , rudolf siguió sirviéndole a jehová', 'y segundo , porque le daremos gloria a nuestro amoroso padre celestial la satisfacción de darle gloria a jehová será mayor que cualquier sacrificio que tengamos que hacer para vestirnos bien', 'la primera cualidad que pablo mencionó es el amor', 'b ¿ quién es la séptima cabeza de la bestia salvaje ?', 'por la misma razón , aunque estemos abiertos a una amplia variedad de diversiones , evitamos todas las que son inmorales , violentas o muy peligrosas , pues violan los principios bíblicos y amenazan nuestra salud física y espiritual', 'en los lugares donde no pudieron hacerlo , le env

In [ ]:
len(en_test_sents)

1000

In [ ]:
import pandas as pd

source_file = "drive/My Drive/mixe-coatlan-es-mco/es-mco-baseline/train-dev-random_es.txt"
target_file = "drive/My Drive/mixe-coatlan-es-mco/es-mco-baseline/train-dev-random_mco.txt"

source = []
target = []
# Recopila los números de línea de la parte de origen para omitir las mismas líneas de la parte de destino. 
skip_lines = []  
with open(source_file) as f:
    for i, line in enumerate(f):
        # Salta la frase que está en el test
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # agrega al corpus solo si no se omitió la fuente correspondiente
        if j not in skip_lines:
            target.append(line.strip())

print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 0/66196 lines since contained in test set.


,source_sentence,target_sentence
0,"sin duda , estos hermanos necesitan que siempr...",seguurë ko tyäˈädë nmëguˈukˈäjtëm yajtëgoyˈäjt...
1,toda escritura es inspirada de dios y provecho...,dios wyinmäˈänymyooy ja jyaˈayëty es tkëxjäˈäy...
2,el amor y unidad que se ve entre los testigos ...,pes diˈibë jesukristë xytyukˈanaˈamëm axtë jaw...


In [ ]:
df.tail(3)

,source_sentence,target_sentence
66194,jehová les dio su espíritu santo a los primero...,jyobaa myëjämooy pënaty pyanëjkxtë kristë mä p...
66195,algo que nos ayudará a comprender los retos im...,tsiptakp nyajtuˈukmujkëmë jäˈäy diˈib wiˈixëmë...
66196,él nos dice en su palabra lo que se busca en l...,"tukˈoybyëtsëm tijaty dios windum ,"


In [ ]:
df_pp = df.drop_duplicates()

df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

In [ ]:
print(len(df_pp))
df_pp.head(2)

66197


,source_sentence,target_sentence
0,"así es , tenemos una lucha contra los gobernan...",ets mët ko yëˈë nmëtsipˈäjtëm nidëgekyëtyë mëj...
1,no nos sentimos atados o cargados al hacer est...,"kyaj nmëkjäˈäwëm ko duˈun nduˈunëm , niˈigyë n..."


In [ ]:
# Instala fuzzy wuzzy para eliminar oraciones "casi duplicadas" en los conjuntos de dev y train

! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np

# restablece el índice del conjunto de entrenamiento después del filtrado anterior 
df_pp.reset_index(drop=False, inplace=True)

# Elimina muestras del conjunto de datos de entrenamiento si "casi se superponen" con conjunto
# de datos de prueba. 


# Función de filtrado. Ajuste el PAD para reducir las coincidencias candidatas a
# una cierta longitud de caracteres de la muestra dada. 
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

#NOTA: esto puede ser lento dependiendo del tamaño del conjunto de entrenamiento. 
scores = []
start_time = time.time()
for idx, row in df_pp.iterrows():
  scores.append(fuzzfilter(row['source_sentence'], list(en_test_sents), 10))
  if idx % 1000 == 0:
    hours, rem = divmod(time.time() - start_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_pp))))

# filtra muestras casi superpuestas
df_pp['scores'] = scores
df_pp = df_pp[df_pp['scores'] < 95]

00:00:00.04 0.00 percent complete
00:00:16.04 1.51 percent complete
00:00:31.05 3.02 percent complete
00:00:46.16 4.53 percent complete
00:01:01.14 6.04 percent complete
00:01:16.01 7.55 percent complete
00:01:31.00 9.06 percent complete
00:01:46.00 10.57 percent complete
00:02:01.00 12.09 percent complete
00:02:15.85 13.60 percent complete
00:02:30.99 15.11 percent complete
00:02:45.95 16.62 percent complete
00:03:00.65 18.13 percent complete
00:03:15.99 19.64 percent complete
00:03:30.77 21.15 percent complete
00:03:45.49 22.66 percent complete
00:04:00.62 24.17 percent complete
00:04:15.29 25.68 percent complete
00:04:30.52 27.19 percent complete
00:04:45.50 28.70 percent complete
00:05:00.38 30.21 percent complete


00:05:15.30 31.72 percent complete
00:05:30.70 33.23 percent complete
00:05:45.70 34.74 percent complete


00:06:01.09 36.26 percent complete


00:06:16.16 37.77 percent complete
00:06:31.23 39.28 percent complete
00:06:45.97 40.79 percent complete
00:07:00.93 42.30 percent complete
00:07:16.07 43.81 percent complete
00:07:31.03 45.32 percent complete
00:07:45.99 46.83 percent complete
00:08:01.11 48.34 percent complete
00:08:16.25 49.85 percent complete
00:08:31.15 51.36 percent complete
00:08:46.24 52.87 percent complete
00:09:01.27 54.38 percent complete
00:09:16.21 55.89 percent complete
00:09:31.43 57.40 percent complete
00:09:46.40 58.92 percent complete
00:10:01.33 60.43 percent complete
00:10:16.16 61.94 percent complete
00:10:31.34 63.45 percent complete
00:10:46.50 64.96 percent complete
00:11:01.53 66.47 percent complete
00:11:16.31 67.98 percent complete
00:11:31.46 69.49 percent complete
00:11:46.92 71.00 percent complete
00:12:02.27 72.51 percent complete
00:12:17.64 74.02 percent complete
00:12:32.82 75.53 percent complete
00:12:47.81 77.04 percent complete
00:13:02.59 78.55 percent complete
00:13:17.54 80.06 pe

00:14:17.55 86.11 percent complete
00:14:33.21 87.62 percent complete
00:14:48.74 89.13 percent complete
00:15:04.48 90.64 percent complete
00:15:19.94 92.15 percent complete
00:15:35.32 93.66 percent complete


00:15:51.56 95.17 percent complete
00:16:06.86 96.68 percent complete
00:16:22.39 98.19 percent complete
00:16:38.14 99.70 percent complete


In [ ]:
print(len(df_pp))

66071


# Para extraer el corpus DEV

In [ ]:
# Esta sección hace la división entre train / dev para los corpus paralelos y luego los guarda como archivos 
#separados. Usamos 1000 dev test y el conjunto de pruebas dado. 

import csv

#1000 alineaciones para el contjunto dev
num_dev_patterns = 1000


# Opcional: corpus minuscula. Esto facilitará la generalización, pero sin el uso de mayúsculas.
if lc:  # Julia: hace que las minusculas sean opcionales
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: conjuntos de prueba ya generados
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

#crea los archivos train.en y train.mco
with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#crea los archivos dev.en y dev.mco    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")


# Verifique dos veces el formato a continuación. No debe haber comillas adicionales ni caracteres extraños. 
! head train.*
! head dev.*

==> train.es <==
así es , tenemos una lucha contra los gobernantes mundiales de esta oscuridad , contra las fuerzas espirituales inicuas en los lugares celestiales
no nos sentimos atados o cargados al hacer estas cosas al contrario , disfrutamos con ellas , y seguro que adán y eva también
deberíamos ayudarlo con paciencia a tener más fe en jehová y a confiar en los hermanos
jehová escuchó la oración de elías y le devolvió el alma , es decir , la vida lea reyes
amamos a jehová por darnos esta gran oportunidad de apoyar su reino y de llevar un mensaje de consuelo a todos los que sufren en este mundo
¿ por qué podemos afirmar que jehová considera importante nuestra elección de un cónyuge ?
al poco tiempo , ya podía mantenerse y era precursor regular
por supuesto , a satanás le encantaría que usted o su familia se alejaran de jehová
¿ ha notado que sus obligaciones aumentan de día en día y le consumen más tiempo ?
ellos cantaron por haberse ofrecido voluntariamente el pueblo , bendigan a j

In [ ]:
ls

dev.es   jw300.es                     JW300_latest_xml_mco.zip  train.es
dev.mco  JW300_latest_xml_es-mco.xml  jw300.mco                 train.mco
drive/   JW300_latest_xml_es.zip      sample_data/


In [ ]:
! cp dev.es /content/drive/MyDrive/mixe-coatlan-es-mco/es-mco-baseline/
! cp dev.mco /content/drive/MyDrive/mixe-coatlan-es-mco/es-mco-baseline/
! cp train.es /content/drive/MyDrive/mixe-coatlan-es-mco/es-mco-baseline/
! cp train.mco /content/drive/MyDrive/mixe-coatlan-es-mco/es-mco-baseline/